In [4]:
from dariah_topics import preprocessing as pre
from dariah_topics import visualization as visual
from dariah_topics import evaluation

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


## Preprocessing

#### Liste mit Dateinamen erzeugen

In [5]:
path_txt = "corpus_txt"
#path_txt = "grenzbote_plain/*/"
#path_txt = "wiki/"

doclist_txt = pre.create_document_list(path_txt)
assert doclist_txt, "No documents found"
doclist_txt[:5]

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/IPython/core/formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator 

['corpus_txt/Doyle_AScandalinBohemia.txt',
 'corpus_txt/Doyle_AStudyinScarlet.txt',
 'corpus_txt/Doyle_TheHoundoftheBaskervilles.txt',
 'corpus_txt/Doyle_TheSignoftheFour.txt',
 'corpus_txt/Howard_GodsoftheNorth.txt']

#####  Liste mit Dokumentenlabels erzeugen - (Funktion wird durch Thorsten's generischere Funktion ersetzt)

In [6]:
doc_labels = list(pre.get_labels(doclist_txt))
doc_labels[:5]

['corpus_txt/Doyle_AScandalinBohemia.txt',
 'corpus_txt/Doyle_AStudyinScarlet.txt',
 'corpus_txt/Doyle_TheHoundoftheBaskervilles.txt',
 'corpus_txt/Doyle_TheSignoftheFour.txt',
 'corpus_txt/Howard_GodsoftheNorth.txt']

#### Corpus laden

In [7]:
corpus_txt = pre.read_from_txt(doclist_txt)

#### Tokenisieren

In [8]:
doc_tokens = [list(pre.tokenize(txt)) for txt in list(corpus_txt)]

#### Create Dictionaries

In [9]:
id_types, doc_ids = pre.create_dictionaries(doc_labels, doc_tokens)

#### Sparse BOW Model

In [10]:
sparse_bow = pre.create_mm(doc_labels, doc_tokens, id_types, doc_ids)

In [11]:
sparse_bow[:5]

0
doc_id token_id   
1      9820      1
       2         1
       12291     1
       12292     3
       6         1

## Topic Modeling with Gensim

##### Saving Sparse BOW

In [12]:
pre.save_bow_mm(sparse_bow, "gensim_txt")

#### Import Market Matrix

In [13]:
from gensim.models import LdaModel
from gensim.corpora import MmCorpus

In [14]:
mm = MmCorpus("gensim_txt.mm")

In [15]:
doc2id = {value : key for key, value in doc_ids.items()}
type2id = {value : key for key, value in id_types.items()}

##### Remove Features

type2id = {value : key for key, value in id_types.items()}
sparse_bow_collapsed = sparse_bow.groupby(sparse_bow.index.get_level_values('token_id')).sum()

sparse_bow_hapax = sparse_bow_collapsed.loc[sparse_bow_collapsed[0] == 1]
hapax = [type2id[key] for key in sparse_bow_hapax.index.get_level_values('token_id')]

len(hapax)

In [16]:
import os.path
basepath = os.path.abspath('.')

with open(os.path.join(basepath, "tutorial_supplementals", "stopwords", "de.txt"), 'r', encoding = 'utf-8') as f: 
    stopword_list = f.read().split('\n')
    
stopword_list = set(stopword_list)

In [17]:
hapax_from_remove = pre.find_hapax(sparse_bow, id_types)
stopwords_from_remove = pre.find_stopwords(sparse_bow, id_types, mfw=75)

#features_to_be_removed = set(hapax_from_remove + stopwords_from_remove)
features_to_be_removed = stopwords_from_remove

sparse_bow_short = pre.remove_features(sparse_bow, id_types, features_to_be_removed)

In [18]:
len(features_to_be_removed)

75

In [19]:
pre.save_bow_mm(sparse_bow_short, "gensim_txt")

In [20]:
mm = MmCorpus("gensim_txt.mm")

#### Convert sparse_bow to list of (doc, tokens) tuples (like doc2bow)

In [21]:
doc2bow_list = []

for doc in sparse_bow_short.index.groupby(sparse_bow_short.index.get_level_values('doc_id')):
    temp = [(token, count) for token, count in zip(sparse_bow_short.loc[doc].index, sparse_bow_short.loc[doc][0])]
    doc2bow_list.append(temp)

In [22]:
len(doc2bow_list)

16

In [23]:
#model = LdaModel(corpus=mm, id2word=type2id, num_topics=60, alpha = "symmetric", passes = 10) #import momentan in visual 
# -> da ich mir noch nicht sicher bin, welche Funktionen in das tm_gensim.py sollen
model = LdaModel(corpus=mm, id2word=type2id, num_topics=20, passes = 10, iterations = 20)

In [24]:
model.get_document_topics(doc2bow_list[1])

[(1, 0.19637437178315331),
 (2, 0.090343075140683918),
 (3, 0.094308762035912083),
 (4, 0.04870776113704553),
 (10, 0.14402773517950188),
 (11, 0.063089538229874467),
 (13, 0.077610164942319246),
 (15, 0.089822382311483345),
 (16, 0.022185781952032801),
 (17, 0.045392065366853834),
 (18, 0.11205460065760847)]

In [25]:
#Anzeige der keywords für topic n
n = 1
topic_nr_x = model.get_topic_terms(n)

topicTerms = [type2id[i[0]] for i in topic_nr_x]
topicTerms

['glowing',
 "beast's",
 'contending',
 'real-true',
 'dollie',
 'cryin',
 'confounded',
 'nursey',
 'charities',
 'tuesday']

In [26]:
#Topic probabilities
topics = model.show_topics(num_topics = 40)
topics

[(0,
  '0.000*"salient" + 0.000*"cryin" + 0.000*"dollie" + 0.000*"rat-holes" + 0.000*"heater" + 0.000*"twice" + 0.000*"cubbing-times" + 0.000*"grabbing" + 0.000*"contending" + 0.000*"charities"'),
 (1,
  '0.005*"glowing" + 0.004*"beast\'s" + 0.004*"contending" + 0.004*"real-true" + 0.004*"dollie" + 0.003*"cryin" + 0.003*"confounded" + 0.003*"nursey" + 0.003*"charities" + 0.003*"tuesday"'),
 (2,
  '0.007*"serpents" + 0.005*"richest" + 0.005*"commented" + 0.005*"treasure-box" + 0.004*"hymnbook" + 0.004*"goings-on" + 0.004*"father\'s" + 0.004*"heater" + 0.004*"exhalation" + 0.004*"contending"'),
 (3,
  '0.012*"system’s" + 0.005*"orifice" + 0.004*"disintegrated" + 0.004*"valuable" + 0.004*"salient" + 0.004*"performed" + 0.003*"treasure-box" + 0.003*"brain-fevered" + 0.003*"fortune" + 0.003*"greetings"'),
 (4,
  '0.009*"glowing" + 0.004*"pyari" + 0.004*"memorably" + 0.004*"tuesday" + 0.004*"grabbing" + 0.004*"propos" + 0.004*"ma" + 0.003*"racial" + 0.003*"kent\'s" + 0.003*"dollie"'),
 (5,
 

In [27]:
import pandas as pd
import regex as re
def gensim2dataframe(model):
    num_topics = model.num_topics
    topics_df = pd.DataFrame(index = range(num_topics), columns= range(10))

    topics = model.show_topics(num_topics)
    
    for topic_dist in topics:    
        idx = topic_dist[0]
        temp = re.findall(r'\"(.+?)\"', topics[idx][1])
        topics_df.loc[idx] = temp
    
    return topics_df

In [28]:
topics_df = gensim2dataframe(model)

In [29]:
doc_topic = topics_df.T
doc_topic

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,salient,glowing,serpents,system’s,glowing,twice,ordinance,persecutors,sitting,twice,turning,slink,contending,fortune,twice,attains,mast,bewilderment,remarkable,hymnbook
1,cryin,beast's,richest,orifice,pyari,fortune,tramped,bluntly,subtle,orifice,fortune,kent's,real-true,civilian,fortune,wall-paper,severing,back-door,antipodes,father's
2,dollie,contending,commented,disintegrated,memorably,treasure-box,malachi,pillows,orifice,beast's,greetings,twice,beast's,cryin,treasure-box,glowing,limitation,repugnance,weakened,monkey-people
3,rat-holes,real-true,treasure-box,valuable,tuesday,greetings,fortune,gods,scourge,cryin,cryin,assumption,heater,shutting,charities,uninteresting,kepler,glowing,charities,real-true
4,heater,dollie,hymnbook,salient,grabbing,salient,smouldered,twenty-three,disintegrated,fortune,scowl,surmounting,propos,peaches,real-true,volunteered,hymnbook,feeble,abortive,twice
5,twice,cryin,goings-on,performed,propos,charities,dependent,omnipotence,field,unequal,unequal,sensitive,twice,astern,dollie,flee,mongoose's,dollie,wave,treasure-box
6,cubbing-times,confounded,father's,treasure-box,ma,cryin,civilian,mocked,marge,salient,trails,fortune,dollie,smouldered,half-visible,faintest,grabbing,gaspee,tarry,half-visible
7,grabbing,nursey,heater,brain-fevered,racial,heater,thicker,filigree,confounded,civilian,incredulity,spacious,volunteered,memorably,cubbing-times,twenty-foot,truffle,half-visible,salient,serpents
8,contending,charities,exhalation,fortune,kent's,trails,oasis,newer,somebodies,greetings,orifice,penetrant,cryin,controlled,heater,grabbing,obedient,hence,ma,dollie
9,charities,tuesday,contending,greetings,dollie,real-true,partake,harm,creak,glowing,hisself,orifice,pyari,countenance,orifice,salient,scorch,twice,unmarried,heater


## Visualisierung 

In [ ]:
'''lda_model = 'out_easy/corpus.lda'
corpus = 'out_easy/corpus.mm'
dictionary = 'out_easy/corpus.dict'
doc_labels = 'out_easy/corpus_doclabels.txt'
interactive  = False

vis = visual.Visualization(lda_model, corpus, dictionary, doc_labels, interactive)'''

##### Create a document-topic matrix (that is a pandas data frame actually)

In [ ]:
doc_topic = visual.create_doc_topic(mm, model, doc_labels)

##### Visualize document-topic matrix

In [ ]:
%matplotlib inline
visual.doc_topic_heatmap(doc_topic)

##### Visualize topic distribution in a single document

In [ ]:
visual.plot_doc_topics(doc_topic, 0)

![success](http://cdn2.hubspot.net/hub/128506/file-446943132-jpg/images/computer_woman_success.jpg)